# 02 – EDA og enkel modell

Vi leser fra SQLite, lager ferskhetsvekter, beregner vektet snitt for partier og simulerer blokkflertall.

In [ ]:
# Oppsett
import sys, os
sys.path.append(os.path.abspath("src"))
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.db import get_conn
from src.model import recency_weights, weighted_average, simulate_majority
from src.utils.party_maps import PARTY_CODES, DEFAULT_BLOCKS


In [ ]:
# Last data og pivoter til bredt format
con = get_conn()
df_polls = pd.read_sql_query('SELECT * FROM polls', con)
df_res = pd.read_sql_query('SELECT * FROM poll_results', con)
wide = df_res.pivot_table(index='poll_id', columns='party', values='percent', aggfunc='first').reset_index()
df = df_polls.merge(wide, left_on='id', right_on='poll_id', how='left')
df['fieldwork_end'] = pd.to_datetime(df['fieldwork_end'], errors='coerce')
df = df.sort_values('fieldwork_end')
df.tail(3)

In [ ]:
# Lag ferskhetsvekter og vektet snitt
df['w_recency'] = recency_weights(df['fieldwork_end'])
avg = weighted_average(df, 'w_recency')
avg.sort_values(ascending=False)

In [ ]:
# Plot vektet gjennomsnitt
plt.figure()
avg[avg.index != 'Andre'].plot(kind='bar')
plt.title('Vektet gjennomsnitt (%) – partier')
plt.ylabel('Prosent')

In [ ]:
# Simuler blokkflertall
maj = simulate_majority(avg)
maj

In [ ]:
# Enkel blokkoppslutning
left = ['Ap','SV','Sp','R','MDG']
right = ['H','FrP','V','KrF']
left_sum = float(avg[left].sum())
right_sum = float(avg[right].sum())
pd.Series({'left_sum': left_sum, 'right_sum': right_sum, 'andre': float(avg.get('Andre', 0.0))})

## Koalisjonsanalyse (skjelett)

Konfigurer en sannsynlighetsmatrise for samarbeid som heuristikk. Dette kan raffineres med ekspertregler eller historikk.

In [ ]:
# Heuristisk koalisjonsmatrise (0–1). Tallene er plassholdere for videre arbeid.
parties = ['Ap','H','FrP','SV','Sp','V','KrF','MDG','R']
import pandas as pd
coalesce = pd.DataFrame(0.0, index=parties, columns=parties)
pref = {
    'V': {'H':0.9, 'KrF':0.7, 'Ap':0.1},
    'KrF': {'H':0.7, 'V':0.7, 'Ap':0.2},
    'Sp': {'Ap':0.8, 'SV':0.6, 'H':0.2},
    'SV': {'Ap':0.9, 'Sp':0.6},
    'MDG': {'Ap':0.7, 'SV':0.6, 'H':0.3},
    'FrP': {'H':0.8, 'V':0.2},
    'Ap': {'SV':0.8, 'Sp':0.7},
    'H': {'V':0.9, 'KrF':0.8, 'FrP':0.6},
    'R': {'SV':0.5, 'Ap':0.4}
}
for a, mp in pref.items():
    for b, v in mp.items():
        coalesce.loc[a, b] = v
coalesce